<a href="https://colab.research.google.com/github/bitturao/dataAnalysis_Retail_Online_Store/blob/main/online_retail_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
!pip install openpyxl


In [81]:
import pandas as pd
from matplotlib import pyplot as plt

In [82]:
!wget https://archive.ics.uci.edu/dataset/352/online+retail.zip

--2025-12-21 01:51:52--  https://archive.ics.uci.edu/dataset/352/online+retail.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174303 (170K) [text/html]
Saving to: ‘online+retail.zip.2’

online+retail.zip.2 100%[===================>] 170.22K  --.-KB/s    in 0.08s   

2025-12-21 01:51:53 (1.99 MB/s) - ‘online+retail.zip.2’ saved [174303/174303]



In [ ]:
!unzip online+retail.zip


Archive:  online+retail.zip
replace Online Retail.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
df1 = pd.read_excel("Online Retail.xlsx", dtype={'InvoiceNo': 'string', 'StockCode':'string', 'Description': 'string', 'Country':'string'})
df1.head(5)

In [ ]:
df1.shape

In [ ]:
df1.info()

### Data Cleaning: Handle Missing Values

In [ ]:
df1.isnull().sum()


In [ ]:
df1[df1.Description.isnull()]

In [ ]:
df1[df1.StockCode=='22139']

In [ ]:
df1[df1.StockCode=='22139'].Description.mode()

In [ ]:
df1[['StockCode','Description']].value_counts()

In [ ]:
most_freq = df1[['StockCode','Description']].value_counts().reset_index()
most_freq

In [ ]:
most_freq[most_freq.StockCode=='22139'].head(1)

In [ ]:
most_freq = most_freq.groupby('StockCode').head(1)
most_freq

In [ ]:
most_freq.columns = ['StockCode', 'freq_Description', 'count']

df2 = df1.merge(most_freq, on='StockCode', how='left')
df2.head()


###Changing all descriptions with mostFrequent descriptions

In [ ]:
df2['Description'] = df2['freq_Description']
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
df2.dropna(subset=['Description'], inplace=True)
df2.isnull().sum() #Dropping the Null Values of Description

In [ ]:
df2.drop(columns=['freq_Description', 'count'], inplace=True)
df2.head()

In [ ]:
df2.describe() # checking for incorrect Data

In [ ]:
df2[df2.Quantity<0] # Maybe refunds

In [ ]:
df3 = df2[(df2.Quantity>0) & (df2.UnitPrice>0)]
df3.describe()

In [ ]:
df3.Quantity.quantile(0.99)

In [ ]:
df3[df3.Quantity>1500] # big Orders

###Feature Engineering: Create New Columns

In [ ]:
df4 = df3.copy()
df4['TotalPrice'] = df4.Quantity * df4.UnitPrice
df4.head()

###Monthly Sales

In [ ]:
df4.info()

In [ ]:
df4['Month'] = df4.InvoiceDate.dt.month
df4.head()

###Visulization & EDA

##Plot Monthly Sales

In [ ]:
monthly_sales = df4.groupby('Month').TotalPrice.sum()
monthly_sales.plot(kind='line',figsize=(10,5), marker = "*")
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.title('Monthly Sales')
plt.grid()
plt.show()

Total sales started rising up in August having a peek in November

#### 2. Top 5 countries

In [ ]:
top_countries = df4.groupby('Country')['TotalPrice'].sum().nlargest(5)
top_countries.sort_values(ascending=True).plot(kind='barh', figsize=(10, 6))
plt.xlabel('Total Sales')
plt.ylabel('Country')
plt.title('Top 5 Countries by Total Sales')
plt.show()

In [ ]:
country_wise_sales = df4.groupby('Country')['TotalPrice'].sum()
total_sales = country_wise_sales.sum()

top_5_countries = country_wise_sales.sort_values(ascending=False).head(5)

top_countries_percentage = (top_5_countries / total_sales) * 100

plt.figure(figsize=(10, 6))
bars = plt.barh(top_5_countries.index, top_countries_percentage.sort_values(ascending= False))
plt.xlabel('Percentage Contribution')
plt.ylabel('Country')
plt.title('Top 5 Countries by Percentage Contribution to Total Sales')
plt.grid(axis='x')

for bar, percentage in zip(bars, top_countries_percentage):
    plt.text(bar.get_width() + 1, bar.get_y() + bar.get_height() / 2,
             f'{percentage:.1f}%)', ha='left', va='center')

plt.show()

In [ ]:
product_wise_sales = df4.groupby('StockCode')['TotalPrice'].sum()
total_sales = product_wise_sales.sum()

top_5_products = product_wise_sales.nlargest(5)

top_products_percentage = (top_5_products / total_sales) * 100

plt.figure(figsize=(10,8))
bars = plt.barh(top_5_products.index, top_products_percentage.sort_values(ascending=False))
plt.xlabel('Percentage Contribution')
plt.ylabel('Product Code')
plt.title('Top 5 Products by Percentage Contribution to Total Sales')
plt.grid(axis='x')

for bar, percentage in zip(bars, top_products_percentage):
    plt.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height() / 2,
             f'{percentage:.1f}%', va='center')

plt.show()

In [ ]:
for stock_code in top_5_products.index:
    product_name = df4[df4['StockCode'] == stock_code]['Description'].iloc[0]
    print(f"{stock_code} ==> {product_name}")
#

In [ ]:
current_date = df4.InvoiceDate.max() + pd.Timedelta(days=1)
current_date #Latest Invoice

In [ ]:
rfm = df4.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (current_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalPrice': 'sum'
})
rfm
rfm.columns = ['Recency', 'Frequncy','Monetary']
rfm.head()

In [ ]:
#verifying the above columns
df4[df4.CustomerID==12346]

In [ ]:
#verifying Monetary
df4[df4.CustomerID==12346].TotalPrice.sum()

In [ ]:
rfm['R-Segment'] = pd.qcut(rfm.Recency, q=5, labels=[5,4,3,2,1])
rfm['F-Segment'] = pd.qcut(rfm.Frequncy, q=5, labels=[1,2,3,4,5])
rfm['M-Segment'] = pd.qcut(rfm.Monetary, q=5, labels=[1,2,3,4,5])
rfm['RFM_Score'] = rfm[['R-Segment','F-Segment','M-Segment']].sum(axis=1)
rfm.head()


In [ ]:
#Customer with highest RFM scores
rfm.sort_values('RFM_Score', ascending=False)

###Customer Churn

In [ ]:
df4.head(3)

In [ ]:
#Create a basket Matrix for association rule mining
customer_last_purchase = df4.groupby('CustomerID')['InvoiceDate'].max()
customer_last_purchase

In [ ]:
current_date

In [ ]:
customer_last_purchase = (current_date - customer_last_purchase).dt.days
customer_last_purchase

In [ ]:
#getting Inactive customers for above 30 days
customer_threshold = 90
churned_customers = customer_last_purchase[customer_last_purchase > customer_threshold]
churned_customers

In [ ]:
print(f"Total Churned Customers: {len(churned_customers)}")
plt.figure(figsize=(10,6))
plt.hist(customer_last_purchase, bins=50, color ='Blue',alpha = 0.7)
plt.axvline(x=customer_threshold, color='red', linestyle='dashed', linewidth=2)
plt.xlabel('Days Since Last Purchase')
plt.ylabel('Number of Customers')
plt.title('Distribution of Days Since Last Purchase for Churned Customers')
plt.show()